In [1]:
from scipy.integrate import quad

In [2]:
import numpy as np
from scipy.stats import norm
from datetime import datetime
import pandas as pd
from scipy.interpolate import interp1d
from scipy.optimize import brentq

# Load Data

In [3]:
df_rates = pd.read_csv("zero_rates_20201201.csv")
df_rates['rate'] = df_rates['rate']/100

rates_interp = interp1d(list(df_rates["days"]), list(df_rates["rate"]))

In [4]:
## load spx and spy data

df_spx_raw = pd.read_csv("SPX_options.csv")
df_spy_raw = pd.read_csv("SPY_options.csv")

In [5]:
df_spx = df_spx_raw.copy()
df_spy = df_spy_raw.copy()

df_spx["strike_price"] = df_spx["strike_price"]/1000
df_spy["strike_price"] = df_spy["strike_price"]/1000

In [6]:
# underlying price of SPX and SPY on 20201201
spx_underlying = 3662.45
spy_underlying = 366.02

In [7]:
df_spx["moneyness"] = "ITM" 

df_spx.loc[(df_spx["cp_flag"] == "C")  & (df_spx["strike_price"] > spx_underlying), "moneyness" ] = "OTM"
df_spx.loc[(df_spx["cp_flag"] == "P")  & (df_spx["strike_price"] < spx_underlying), "moneyness" ] = "OTM"

df_spy["moneyness"] = "ITM" 

df_spy.loc[(df_spy["cp_flag"] == "C")  & (df_spy["strike_price"] > spy_underlying), "moneyness" ] = "OTM"
df_spy.loc[(df_spy["cp_flag"] == "P")  & (df_spy["strike_price"] < spy_underlying), "moneyness" ] = "OTM"

In [8]:
#using average of best bid and best offer to get mid price, which will be used to calculate implied volatility

df_spx["mid_price"] = (df_spx["best_bid"] + df_spx["best_offer"])/2
df_spy["mid_price"] = (df_spy["best_bid"] + df_spy["best_offer"])/2

In [9]:
#change expiry to datetime format
df_spx["expiry"] = pd.to_datetime(df_spx["exdate"], format = "%Y%m%d")
df_spy["expiry"] = pd.to_datetime(df_spy["exdate"], format = "%Y%m%d")

In [10]:
#calculate days from expiry for each SPX and SPY options

df_spx["days"] = df_spx["expiry"] -  datetime(2020,12,1)
df_spx["days"] = df_spx["days"].dt.days 
df_spx["T"] = df_spx["days"] / 365

df_spy["days"] = df_spy["expiry"] -  datetime(2020,12,1)
df_spy["days"] = df_spy["days"].dt.days 
df_spy["T"] = df_spy["days"] / 365

In [11]:
#calculate rates by interpolating using "zero rates 20201201.csv"

df_spx["r"] = df_spx["days"].apply(lambda x: rates_interp([x])[0])
df_spy["r"] = df_spy["days"].apply(lambda x: rates_interp([x])[0])

In [12]:
#calculate forward price at expiry F(T)
df_spx["F"] = spx_underlying * np.power(np.e, ((df_spx["r"]) * (df_spx["T"])))
df_spy["F"] = spy_underlying * np.power(np.e, ((df_spy["r"]) * (df_spy["T"])))

In [13]:
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)

def BachelierCall(S, K, r, sigma, T):
    d1 = (S-K)/(sigma*np.sqrt(T))
    return np.exp(-r*T)*((S-K)*norm.cdf(d1)+sigma*np.sqrt(T)*norm.pdf(d1))

def BachelierPut(S, K, r, sigma, T):
    return BachelierCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)

def calc_implied_vol(S,K, r, price, T, flag, moneyness, model):
    "calculate implied volality for OTM only"
    if moneyness != "OTM":
        return np.nan
    else:
        impliedvol = impliedVolatility(S, K, r, price, T, flag, model)
        #print(impliedvol)
        return impliedvol
    
def impliedVolatility(S, K, r, price, T, payoff, model):
    "return implied volatility using black scholes given price"
    #try:
    if (payoff.lower() == 'c'):
        if (model.lower() == 'blackscholes'):
            impliedVol = brentq(lambda x: price - BlackScholesCall(S, K, r, x, T), -1e-12, 10)
        else:
            impliedVol = brentq(lambda x: price - BachelierCall(S, K, r, x, T), -1e-12, 1e10)
    elif (payoff.lower() == 'p'):
        if (model.lower() == 'blackscholes'):
            impliedVol = brentq(lambda x: price - BlackScholesPut(S, K, r, x, T), -1e-12, 10)
        else:
            impliedVol = brentq(lambda x: price - BachelierPut(S, K, r, x, T), -1e-12, 1e10)
    else:
        raise NameError('Payoff type not recognized')
    #except Exception:
    #    print('ex')
    #    impliedVol = np.nan

    return impliedVol

In [14]:
#calculate implied vol for OTM only

df_spx["impliedvol_blackscholes"] = df_spx.apply(lambda r: calc_implied_vol(
                        spx_underlying,
                        r["strike_price"],
                        r["r"],
                        r["mid_price"],
                        r['T'],
                        r["cp_flag"],
                        r["moneyness"],'blackscholes'), axis = 1)
df_spy["impliedvol_blackscholes"] = df_spy.apply(lambda r: calc_implied_vol(
                        spy_underlying,
                        r["strike_price"],
                        r["r"],
                        r["mid_price"],
                        r['T'],
                        r["cp_flag"],
                        r["moneyness"],'blackscholes'), axis = 1)
df_spx["impliedvol_bachelier"] = df_spx.apply(lambda r: calc_implied_vol(
                        spx_underlying,
                        r["strike_price"],
                        r["r"],
                        r["mid_price"],
                        r['T'],
                        r["cp_flag"],
                        r["moneyness"],'bachelier'), axis = 1)
df_spy["impliedvol_bachelier"] = df_spy.apply(lambda r: calc_implied_vol(
                        spy_underlying,
                        r["strike_price"],
                        r["r"],
                        r["mid_price"],
                        r['T'],
                        r["cp_flag"],
                        r["moneyness"],'bachelier'), axis = 1)

In [15]:
#only interested in the OTM options to plot volatility smile

df_spx_otm = df_spx[df_spx["moneyness"] == "OTM"]
df_spy_otm = df_spy[df_spy["moneyness"] == "OTM"]

In [16]:
def function_h(s):
    return s**(1/3) + 1.5 * np.log(s) + 10

#def function_h1(s):
#    return (1/3)*s**(-2/3) + (1.5/s)

def function_h2(s):
    return -(2/9)*s**(-5/3) - (1.5/(s**2))

# static replication for SPX (using Jan expiry)

In [17]:
#df_spx_dec = df_spx_otm[df_spx_otm["exdate"] == 20201218].reset_index(drop = True)
df_spx_jan = df_spx_otm[df_spx_otm["exdate"] == 20210115].reset_index(drop = True)
#df_spx_feb = df_spx_otm[df_spx_otm["exdate"] == 20210219].reset_index(drop = True)

In [18]:
T = df_spx_jan["T"].iloc[0]
T

0.1232876712328767

In [19]:
r = df_spx_jan["r"].iloc[0]
#example of interpolating rate for a given day
#rates_interp = interp1d(list(df_rates["days"]), list(df_rates["rate"]))
#r = rates_interp([(datetime(2021, 1 ,15) - datetime(2020, 12 ,1)).days])[0]
r

0.002051075555555556

In [20]:
S = spx_underlying
S

3662.45

In [21]:
F = df_spx_jan["F"].iloc[0]
#F =  S * np.exp(r*T)
F

3663.3762493669747

## Black Scholes

In [22]:
#get implied vol, midprice and strike of OTM put/call nearest to underlying price (ATM)

df_spx_jan_c = df_spx_jan[df_spx_jan["cp_flag"] == "C"]
call_otm_implied_vol_jan = df_spx_jan_c.loc[(df_spx_jan_c["strike_price"] - spx_underlying).idxmin()]["impliedvol_blackscholes"]
call_otm_midprice_jan = df_spx_jan_c.loc[(df_spx_jan_c["strike_price"] - spx_underlying).idxmin()]["mid_price"]
call_otm_K_jan = df_spx_jan_c.loc[(df_spx_jan_c["strike_price"] - spx_underlying).idxmin()]["strike_price"]

df_spx_jan_p = df_spx_jan[df_spx_jan["cp_flag"] == "P"]
put_otm_implied_vol_jan = df_spx_jan_p.loc[(spx_underlying - df_spx_jan_p["strike_price"] ).idxmin()]["impliedvol_blackscholes"]
put_otm_midprice_jan = df_spx_jan_p.loc[(spx_underlying - df_spx_jan_p["strike_price"] ).idxmin()]["mid_price"]
put_otm_K_jan = df_spx_jan_p.loc[(spx_underlying - df_spx_jan_p["strike_price"] ).idxmin()]["strike_price"]

In [28]:
#use interpolation of nearest otm put implied vol and nearest otm call implied vol to get sigma for BS

sigmaBS = (spx_underlying - put_otm_K_jan) / (call_otm_K_jan - put_otm_K_jan) * (call_otm_implied_vol_jan - put_otm_implied_vol_jan) + put_otm_implied_vol_jan
sigmaBS

0.1849096526277256

#### For payoff function S^1/3 + 1.5 log(S) + 10

In [29]:
def BScall_payoff1(S, K, r, sigma, T):
    price = BlackScholesCall(S, K, r, sigma, T) * function_h2(K)
    return price


def BSput_payoff1(S, K, r, sigma, T):
    price = BlackScholesPut(S, K, r, sigma, T) * function_h2(K)
    return price

In [30]:
I_put = quad(lambda x: BSput_payoff1(S, x, r, sigmaBS, T), 1e-6, F)

I_call = quad(lambda x: BScall_payoff1(S, x, r, sigmaBS, T), F, 5400)

In [31]:
np.exp(r*T)*function_h(F) + I_put[0] + I_call[0]

37.72397660228189

#### for payoff function: model free integrated variance

In [32]:
def BScall_payoff2(S, K, r, sigma, T):
    price = BlackScholesCall(S, K, r, sigma, T)  / (K**2)
    return price


def BSput_payoff2(S, K, r, sigma, T):
    price = BlackScholesPut(S, K, r, sigma, T)  / (K**2)
    return price

In [111]:
I_put = quad(lambda x: BSput_payoff2(S, x, r, sigmaBS, T), 1e-6, F)

I_call = quad(lambda x: BScall_payoff2(S, x, r, sigmaBS, T), F, 5400)

In [112]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
E_var

0.00421540022888456

## Bachelier

In [87]:
#get implied vol, midprice and strike of OTM put/call nearest to underlying price (ATM)

df_spx_jan_c = df_spx_jan[df_spx_jan["cp_flag"] == "C"]
call_otm_implied_vol_jan = df_spx_jan_c.loc[(df_spx_jan_c["strike_price"] - spx_underlying).idxmin()]["impliedvol_bachelier"]
call_otm_midprice_jan = df_spx_jan_c.loc[(df_spx_jan_c["strike_price"] - spx_underlying).idxmin()]["mid_price"]
call_otm_K_jan = df_spx_jan_c.loc[(df_spx_jan_c["strike_price"] - spx_underlying).idxmin()]["strike_price"]

df_spx_jan_p = df_spx_jan[df_spx_jan["cp_flag"] == "P"]
put_otm_implied_vol_jan = df_spx_jan_p.loc[(spx_underlying - df_spx_jan_p["strike_price"] ).idxmin()]["impliedvol_bachelier"]
put_otm_midprice_jan = df_spx_jan_p.loc[(spx_underlying - df_spx_jan_p["strike_price"] ).idxmin()]["mid_price"]
put_otm_K_jan = df_spx_jan_p.loc[(spx_underlying - df_spx_jan_p["strike_price"] ).idxmin()]["strike_price"]

In [88]:
#use interpolation of nearest otm put implied vol and nearest otm call implied vol to get sigma for Bachelier

sigmaBachelier = (spx_underlying - put_otm_K_jan) / (call_otm_K_jan - put_otm_K_jan) * (call_otm_implied_vol_jan - put_otm_implied_vol_jan) + put_otm_implied_vol_jan
sigmaBachelier

680.4947335307122

#### For payoff function S^1/3 + 1.5 log(S) + 10

In [105]:
#### For payoff function S^1/3 + 1.5 log(S) + 10
def callintegrand(S, K, r, sigma, T):
    price = BachelierCall(S, K, r, sigma, T) * function_h2(K)
    return price

def putintegrand(S, K, r, sigma, T):
    price = BachelierPut(S, K, r, sigma, T) * function_h2(K)
    return price

I_put = quad(lambda x: putintegrand(S, x, r, sigmaBachelier, T), 1e-6, F)
I_call = quad(lambda x: callintegrand(S, x, r, sigmaBachelier, T), F, 5400)

C:\Users\Statistician\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  # Remove the CWD from sys.path while we load stuff.


In [106]:
np.exp(r*T)*function_h(F) + I_put[0] + I_call[0]

37.72214118956419

#### for payoff function: model free integrated variance

In [107]:
def callintegrand2(S, K, r, sigma, T):
    price = BachelierCall(S, K, r, sigma, T)  / (K**2)
    return price

def putintegrand2(S, K, r, sigma, T):
    price = BachelierPut(S, K, r, sigma, T)  / (K**2)
    return price

I_put = quad(lambda x: BSput_payoff2(S, x, r, sigmaBS, T),  1e-6, F)
I_call = quad(lambda x: BScall_payoff2(S, x, r, sigmaBS, T), F, 5400)

In [108]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
E_var

0.00421540022888456

## SABR

Calibrated SABR model parameters for 20210115 expiry: alpha = 1.817, beta = 0.7, rho = -0.404, nu = 2.790

In [42]:
alpha = 1.817
beta = 0.7
rho = -0.404
nu = 2.790

In [43]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [44]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


#### For payoff function S^1/3 + 1.5 log(S) + 10

In [45]:
def sabrcallintegrand_payoff1(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) * function_h2(K)
    return price


def sabrputintegrand_payoff1(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) * function_h2(K)
    return price

In [46]:
I_put = quad(lambda x: sabrputintegrand_payoff1(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)

I_call = quad(lambda x: sabrcallintegrand_payoff1(x, S, r, T, alpha, beta, rho, nu), F, 5400)

In [47]:
np.exp(r*T)*function_h(F) + I_put[0] + I_call[0]

37.71946650705815

#### for payoff function: model free integrated variance

In [115]:
def sabrcallintegrand_payoff2(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / (K**2)
    return price


def sabrputintegrand_payoff2(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / (K**2)
    return price

In [118]:
I_put = quad(lambda x: sabrputintegrand_payoff2(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
I_call = quad(lambda x: sabrcallintegrand_payoff2(x, S, r, T, alpha, beta, rho, nu), F, 5400)

In [119]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
E_var

0.00634467674824901